In [6]:
import string

def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	doc = doc.replace('--', ' ')
	tokens = doc.split()
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	tokens = [word for word in tokens if word.isalpha()]
	tokens = [word.lower() for word in tokens]
	return tokens

def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	seq = tokens[i-length:i]
	line = ' '.join(seq)
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner t
['i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'ch

In [7]:
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

# load doc into memory
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

vocab_size = len(tokenizer.word_index) + 1

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X, y, batch_size=128, epochs=20)

model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Train on 118631 samples
Epoch 1/20
118631/118631 [==============================] - 86s 723us/sample - loss: 3.8491 - accuracy: 0.2293
Epoch 2/20
118631/118631 [==============================] - 85s 715us/sample - loss: 3.8197 - accuracy: 0.2330
Epoch 3/20
118631/118631 [==============================] - 85s 719us/sample - loss: 3.7913 - accuracy: 0.2358
Epoch 4/20
118631/118631 [==============================] - 85s 718us/sample - loss: 3.7628 - accuracy: 0.2381
Epoch 5/20
118631/118631 [==============================] - 85s 718us/sample - loss: 3.7359 - accuracy: 0.2425
Epoch 6/20
118631/118631 [==============================] - 86s 722us/sample - loss: 3.7083 - accuracy: 0.2450
Epoch 7/20
118631/118631 [==============================] - 85s 716us/sample - loss: 3.6816 - accuracy: 0.2474
Epoch 8/20
118631/118631 [==============================] - 86s 721us/sample - loss: 3.6544 - accuracy: 0.2510
Epoch 9/20
118631/118631 [==============================] - 86s 723us/sample - loss: 3.6

In [14]:
from random import randint
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	for _ in range(n_words):
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		yhat = model.predict_classes(encoded, verbose=0)
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

model = load_model('model.h5')

tokenizer = load(open('tokenizer.pkl', 'rb'))

seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

print("Generated")
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

and use of women and children is to follow the path on which we originally started when we said that the men were to be the guardians and watchdogs of the herd true let us further suppose the birth and education of our women to be subject to similar or nearly

GENER
conceits that is the same principle and borne by the same time and the other virtues is the other of all sorts of materials and assemblies and the other virtues of the state and the echo of the soul and the other of the stronger and the other of the
